<a href="https://colab.research.google.com/github/Redislabs-Solution-Architects/Redis-Workshops/blob/main/08-Semantic_Router/08-Semantic_Router.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Routing

RedisVL provides a `SemanticRouter` interface to utilize Redis' built-in search & aggregation in order to perform
KNN-style classification over a set of `Route` references to determine the best match.

This notebook will go over how to use Redis as a Semantic Router for your applications

In [1]:
# Install the requirements
!pip install -q redisvl sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.1 MB/s eta 0:00:00


### Install Redis Stack locally
This installs the Redis Stack database for local usage if you cannot or do not want to create a Redis Cloud database, and it installs the redis-cli that will be used for checking database connectivity, etc.

In [2]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes


deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


### Setup Redis connection
Note - Replace Redis connection values if using a Redis Cloud instance!

In [3]:
import os

REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
# Replace values with your own if using Redis Cloud instance
#REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=18374
#REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# Shortcut for redis-cli $REDIS_CONN command
# If SSL is enabled on the endpoint add --tls
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

## Define the Routes

Below we define 3 different routes. One for `technology`, one for `sports`, and
another for `entertainment`. Now for this example, the goal here is
surely topic "classification". But you can create routes and references for
almost anything.

Each route has a set of references that cover the "semantic surface area" of the
route. The incoming query from a user needs to be semantically similar to one or
more of the references in order to "match" on the route.

In [4]:
from redisvl.extensions.router import Route


# Define routes for the semantic router
technology = Route(
    name="technology",
    references=[
        "what are the latest advancements in AI?",
        "tell me about the newest gadgets",
        "what's trending in tech?"
    ],
    metadata={"category": "tech", "priority": 1}
)

sports = Route(
    name="sports",
    references=[
        "who won the game last night?",
        "tell me about the upcoming sports events",
        "what's the latest in the world of sports?",
        "sports",
        "basketball and football"
    ],
    metadata={"category": "sports", "priority": 2}
)

entertainment = Route(
    name="entertainment",
    references=[
        "what are the top movies right now?",
        "who won the best actor award?",
        "what's new in the entertainment industry?"
    ],
    metadata={"category": "entertainment", "priority": 3}
)


## Initialize the SemanticRouter

``SemanticRouter`` will automatically create an index within Redis upon initialization for the route references. By default, it uses the `HFTextVectorizer` to
generate embeddings for each route reference.

In [5]:
from redisvl.extensions.router import SemanticRouter
from redisvl.utils.vectorize import HFTextVectorizer

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Initialize the SemanticRouter
router = SemanticRouter(
    name="topic-router",
    vectorizer=HFTextVectorizer(),
    routes=[technology, sports, entertainment],
    redis_url=REDIS_URL,
    overwrite=True # Blow away any other routing index with this name
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


13:59:49 numexpr.utils INFO   NumExpr defaulting to 2 threads.
13:59:54 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: cpu
13:59:54 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
router.vectorizer

HFTextVectorizer(model='sentence-transformers/all-mpnet-base-v2', dims=768)

In [7]:
# look at the index specification created for the semantic router
!rvl index info -i topic-router



Index Information:
╭──────────────┬────────────────┬──────────────────┬─────────────────┬────────────╮
│ Index Name   │ Storage Type   │ Prefixes         │ Index Options   │   Indexing │
├──────────────┼────────────────┼──────────────────┼─────────────────┼────────────┤
│ topic-router │ HASH           │ ['topic-router'] │ []              │          0 │
╰──────────────┴────────────────┴──────────────────┴─────────────────┴────────────╯
Index Fields:
╭────────────┬─────────────┬────────┬────────────────┬────────────────┬────────────────┬────────────────┬────────────────┬────────────────┬─────────────────┬────────────────╮
│ Name       │ Attribute   │ Type   │ Field Option   │ Option Value   │ Field Option   │ Option Value   │ Field Option   │   Option Value │ Field Option    │ Option Value   │
├────────────┼─────────────┼────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┤
│ route_name │ route

## Simple routing

In [8]:
# Query the router with a statement
route_match = router("Can you tell me about the latest in artificial intelligence?")
route_match

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

RouteMatch(name='technology', distance=0.119615375996)

In [9]:
# Query the router with a statement and return a miss
route_match = router("are aliens real?")
route_match

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

RouteMatch(name=None, distance=None)

In [10]:
# Toggle the runtime distance threshold
route_match = router("Which basketball team will win the NBA finals?", distance_threshold=0.7)
route_match

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

RouteMatch(name='sports', distance=0.554210186005)

We can also route a statement to many routes and order them by distance:

In [11]:
# Perform multi-class classification with route_many() -- toggle the max_k and the distance_threshold
route_matches = router.route_many("Lebron James", distance_threshold=1.0, max_k=3)
route_matches

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[RouteMatch(name='sports', distance=0.758580756187),
 RouteMatch(name='entertainment', distance=0.812423805396),
 RouteMatch(name='technology', distance=0.884235362212)]

In [12]:
# Toggle the aggregation method -- note the different distances in the result
from redisvl.extensions.router.schema import DistanceAggregationMethod

route_matches = router.route_many("Lebron James", aggregation_method=DistanceAggregationMethod.min, distance_threshold=1.0, max_k=3)
route_matches

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[RouteMatch(name='sports', distance=0.663254261017),
 RouteMatch(name='entertainment', distance=0.712985336781),
 RouteMatch(name='technology', distance=0.832674622536)]

Note the different route match distances. This is because we used the `min` aggregation method instead of the default `avg` approach.

## Update the routing config

In [13]:
from redisvl.extensions.router import RoutingConfig

router.update_routing_config(
    RoutingConfig(distance_threshold=1.0, aggregation_method=DistanceAggregationMethod.min, max_k=3)
)

In [14]:
route_matches = router.route_many("Lebron James")
route_matches

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[RouteMatch(name='sports', distance=0.663254261017),
 RouteMatch(name='entertainment', distance=0.712985336781),
 RouteMatch(name='technology', distance=0.832674622536)]

## Router serialization

In [15]:
router.to_dict()

{'name': 'topic-router',
 'routes': [{'name': 'technology',
   'references': ['what are the latest advancements in AI?',
    'tell me about the newest gadgets',
    "what's trending in tech?"],
   'metadata': {'category': 'tech', 'priority': '1'}},
  {'name': 'sports',
   'references': ['who won the game last night?',
    'tell me about the upcoming sports events',
    "what's the latest in the world of sports?",
    'sports',
    'basketball and football'],
   'metadata': {'category': 'sports', 'priority': '2'}},
  {'name': 'entertainment',
   'references': ['what are the top movies right now?',
    'who won the best actor award?',
    "what's new in the entertainment industry?"],
   'metadata': {'category': 'entertainment', 'priority': '3'}}],
 'vectorizer': {'type': 'hf',
  'model': 'sentence-transformers/all-mpnet-base-v2'},
 'routing_config': {'distance_threshold': 1.0,
  'max_k': 3,
  'aggregation_method': 'min'}}

In [16]:
router2 = SemanticRouter.from_dict(router.to_dict(), redis_url="redis://localhost:6379")

assert router2 == router

14:00:04 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: cpu
14:00:04 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

14:00:05 redisvl.index.index INFO   Index already exists, not overwriting.


In [17]:
router.to_yaml("router.yaml", overwrite=True)
!cat router.yaml

name: topic-router
routes:
- name: technology
  references:
  - what are the latest advancements in AI?
  - tell me about the newest gadgets
  - what's trending in tech?
  metadata:
    category: tech
    priority: '1'
- name: sports
  references:
  - who won the game last night?
  - tell me about the upcoming sports events
  - what's the latest in the world of sports?
  - sports
  - basketball and football
  metadata:
    category: sports
    priority: '2'
- name: entertainment
  references:
  - what are the top movies right now?
  - who won the best actor award?
  - what's new in the entertainment industry?
  metadata:
    category: entertainment
    priority: '3'
vectorizer:
  type: hf
  model: sentence-transformers/all-mpnet-base-v2
routing_config:
  distance_threshold: 1.0
  max_k: 3
  aggregation_method: min


In [18]:
router3 = SemanticRouter.from_yaml("router.yaml", redis_url="redis://localhost:6379")

assert router3 == router2 == router

14:00:05 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: cpu
14:00:05 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

14:00:06 redisvl.index.index INFO   Index already exists, not overwriting.


## Clean up the router

In [19]:
# Use clear to flush all routes from the index
router.clear()

In [20]:
# Use delete to clear the index and remove it completely
router.delete()